<a href="https://colab.research.google.com/github/keonju2/sol_ch/blob/main/0%EC%A0%9C%EA%B1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!pip install -r requirements.txt # install dependencies
!pip install -q roboflow

import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 11142, done.
remote: Total 11142 (delta 0), reused 0 (delta 0), pack-reused 11142
Receiving objects: 100% (11142/11142), 11.14 MiB | 32.40 MiB/s, done.
Resolving deltas: 100% (7699/7699), done.
/content/yolov5
     |████████████████████████████████| 596 kB 5.1 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 67 kB 3.0 MB/s 
     |████████████████████████████████| 178 kB 30.8 MB/s 
     |████████████████████████████████| 138 kB 40.8 MB/s 
     |████████████████████████████████| 1.1 MB 47.6 MB/s 
     |████████████████████████████████| 145 kB 59.8 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cola

In [3]:
from zipfile import ZipFile
import os

def unzip(source_file, dest_path):
    with ZipFile(source_file, 'r') as zf:
        zipInfo = zf.infolist()
        for member in zipInfo:
            try:
                member.filename = member.filename.encode('cp437').decode('euc-kr', 'ignore')
                zf.extract(member, dest_path)
            except:
                raise Exception('what?!')

In [4]:
!pip install -q wandb
import wandb
wandb.login()

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 181 kB 48.3 MB/s 
     |████████████████████████████████| 144 kB 48.4 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
file_names = '/content/gdrive/MyDrive/custom_dataset.zip'
unzip(file_names,'/content/yolov5/custom_dataset')

In [6]:
import shutil
import os

file_source = '/content/yolov5/custom_dataset/custom_dataset/train/labels'
file_dest = '/content/yolov5/custom_dataset/custom_dataset/val/labels'
file_dest2 = '/content/yolov5/data/detect/labels'

get_file = os.listdir(file_source)
for i in get_file:
  if 'Q_0' in i:
    move_file = file_source + '/' + i
    shutil.move(move_file, file_dest2)

In [7]:
file_source = '/content/yolov5/custom_dataset/custom_dataset/test/labels'
file_dest = '/content/yolov5/custom_dataset/custom_dataset/val/labels'
file_dest2 = '/content/yolov5/data/detect/labels'

get_file = os.listdir(file_source)
for i in get_file:
  if 'Q_0' in i or 'Q_1' in i :
    move_file = file_source + '/' + i
    shutil.move(move_file, file_dest2)
  if len(i) == 6:
    move_file = file_source + '/' + i
    shutil.move(move_file, file_dest)

In [8]:
file_name = '/content/gdrive/MyDrive/sc_data_zip/Training/image'
train_list = os.listdir(file_name)
print(train_list)
for i in train_list:
  file_names = file_name+'/'+i
  if '실제도로환경' in i:
      unzip(file_names,'/content/yolov5/custom_dataset/custom_dataset/train/images')

  elif 'keypoint' in file_names:
      unzip(file_names,'/content/yolov5/data/detect/images')
  else:
      unzip(file_names,'/content/yolov5/custom_dataset/custom_dataset/train/images')


['[원천]bbox(실제도로환경).zip', '[원천]bbox(통제환경).zip', '[원천]keypoint(준통제환경).zip']


In [9]:
file_name = '/content/gdrive/MyDrive/sc_data_zip/Validation/image'
train_list = os.listdir(file_name)
print(train_list)

for i in train_list:
  file_names = file_name+'/'+i
  if '[원천]bbox(실제도로환경).zip' in i:
      unzip(file_names,'/content/yolov5/custom_dataset/custom_dataset/test/images')
  elif 'keypoint' in file_names:
      unzip(file_names,'/content/yolov5/data/detect/images')
  else:
      unzip(file_names,'/content/yolov5/custom_dataset/custom_dataset/val/images')


['[원천]bbox(실제도로환경).zip', '[원천]bbox(통제환경).zip', '[원천]keypoint(준통제환경).zip']


In [10]:
import yaml

make_yaml = {
             "train" : "/content/yolov5/custom_dataset/custom_dataset/train",
            "test": "/content/yolov5/custom_dataset/custom_dataset/test",
            "val": "/content/yolov5/custom_dataset/custom_dataset/val",

             "nc" : 4,
             "names": ["left_open_eye","left_closed_eye", "right_open_eye","right_closed_eye"]
}
with open('/content/yolov5/data/custom_dataset.yaml', 'w') as outfile:
    yaml.dump(make_yaml, outfile, default_flow_style=False)

In [11]:
!python train.py --img 640 --cfg yolov5s.yaml --hyp /content/yolov5/data/hyps/hyp.scratch-high.yaml --batch 32 --epochs 3 --data /content/yolov5/data/custom_dataset.yaml --weights yolov5s.pt --workers 24

wandb: Currently logged in as: keonju2 (use `wandb login --relogin` to force relogin)
train: weights=yolov5s.pt, cfg=yolov5s.yaml, data=/content/yolov5/data/custom_dataset.yaml, hyp=/content/yolov5/data/hyps/hyp.scratch-high.yaml, epochs=3, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=24, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-11-g63ddb6f torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05,

In [12]:
save_path = '/content/gdrive/MyDrive/result/fourth'
save_file_path = '/content/yolov5/runs/train'

shutil.move(save_file_path, save_path)

'/content/gdrive/MyDrive/result/fourth'

In [13]:
!python detect.py --source /content/yolov5/data/detect/images/Q_001_30_M_01_M0_G0_C0 --weights /content/gdrive/MyDrive/result/fourth/exp/weights/best.pt --conf 0.25 --name exp

detect: weights=['/content/gdrive/MyDrive/result/fourth/exp/weights/best.pt'], source=/content/yolov5/data/detect/images/Q_001_30_M_01_M0_G0_C0, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-11-g63ddb6f torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 213 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
image 1/25 /content/yolov5/data/detect/images/Q_001_30_M_01_M0_G0_C0/Q_001_30_M_01_M0_G0_C0_01.jpg: 640x384 1 left_open_eye, 1 left_closed_eye, 1 right_open_eye, 1 right_closed_eye, Done. (0.016s)
image 2/25 /content/yolov5/data/detect/images/Q_001_30_M_01_M0_G0_C0/Q_001_30_M_01_M0_G0_C0_02.jpg: 6

In [14]:
!python models/tf.py --weights /content/gdrive/MyDrive/result/fourth/exp/weights/best.pt

tf: weights=/content/gdrive/MyDrive/result/fourth/exp/weights/best.pt, imgsz=[640, 640], batch_size=1, dynamic=False
Model Summary: 270 layers, 7030417 parameters, 0 gradients, 15.9 GFLOPs
2022-02-28 07:46:38.928694: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    115712  models.common.C3     

In [17]:
save_path = '/content/gdrive/MyDrive/result/fourth/detect'
save_file_path = '/content/yolov5/runs/detect/exp'

shutil.move(save_file_path, save_path)

'/content/gdrive/MyDrive/result/fourth/detect'

In [21]:
!python export.py --weights /content/gdrive/MyDrive/result/fourth/exp/weights/best.pt --include tflite

export: data=data/coco128.yaml, weights=['/content/gdrive/MyDrive/result/fourth/exp/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, train=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['tflite']
YOLOv5 🚀 v6.1-11-g63ddb6f torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 213 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from /content/gdrive/MyDrive/result/fourth/exp/weights/best.pt with output shape (1, 25200, 9) (14.4 MB)

TensorFlow SavedModel: starting export with tensorflow 2.8.0...

                 from  n    params  module                                  arguments                     
2022-02-28 07:53:00.946172: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
